## Setting Up

In [16]:
from openai import OpenAI
import requests
from tenacity import retry, wait_random_exponential, stop_after_attempt
from termcolor import colored

api_key=""

# GPT_MODEL_3 = "gpt-3.5-turbo-0125"
GPT_MODEL_3 = "gpt-3.5-turbo-1106"
GPT_MODEL_3_LONG = "gpt-4-32k-0314"
GPT_MODEL_4 = "gpt-4-0613"

client = OpenAI(api_key=api_key)

## Utilities

First let's define a few utilities for making calls to the Chat Completions API and for maintaining and keeping track of the conversation state.

In [7]:
def get_response(messages, functions, model, function_call):
    try:
        # messages = []
        # messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
        # messages.append({"role": "user", "content": user_prompt})
        # messages = [{'role': 'user', 'content': user_prompt}]
        response = client.chat.completions.create(
            model = model,
            messages = messages,
            functions = functions,
            function_call = function_call
        )
        # return response.choices[0].message.function_call.arguments
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")

## Tools

In [3]:
from function_calling.functions import functions

# Start Chating

## Testing Functions

### Trend Detection

In [4]:
# messages = []
# messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
# messages.append({"role": "user", "content": "What's the trend of NQ"})
# chat_response = get_openai_response(
#     messages=messages, functions=tools, model=GPT_MODEL_3, tool_choice="auto"
# )
# assistant_message = chat_response.json()["choices"][0]["message"]
# messages.append(assistant_message)
# assistant_message

In [9]:
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "What is the trend of NQ stock?"})
chat_response = get_response(
    messages, functions, GPT_MODEL_3, "auto"
    # messages, functions=functions, tool_choice="auto", model=GPT_MODEL_3
)
assistant_message = chat_response.choices[0].message
messages.append(assistant_message)
assistant_message

ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{\n  "symbol": "NQ"\n}', name='detect_trend'), tool_calls=None)

In [10]:
messages.append({"role": "user", "content": "over the last month"})
chat_response = get_response(
    messages, functions, GPT_MODEL_3, "auto"
)
chat_response.choices[0]

Choice(finish_reason='function_call', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='from datetime import datetime, timedelta\n\nend_date = datetime.now()\nstart_date = end_date - timedelta(days=30)\n\ntimerange = [int(start_date.timestamp()), int(end_date.timestamp())]\n\n{\n  "symbol": "NQ",\n  "timerange": timerange\n}', name='python'), tool_calls=None))

In [1]:
# in this cell we force the model to use get_n_day_weather_forecast
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "What is the trend of NQ stock over the last month?"})
chat_response = get_response(
    messages, functions, GPT_MODEL_3, "auto"
)
print(chat_response.choices[0].message.content)

NameError: name 'get_response' is not defined

In [12]:
chat_response

ChatCompletion(id='chatcmpl-94AQRnMJlcCEgkbmiwEaNNj3EMvy0', choices=[Choice(finish_reason='function_call', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{\n  "symbol": "NQ",\n  "timerange": "last month"\n}', name='detect_trend'), tool_calls=None))], created=1710780415, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=26, prompt_tokens=1439, total_tokens=1465))

In [6]:
user_prompt = "What is the trend of NQ stock over the last month?"

output = get_response(user_prompt, functions, model=GPT_MODEL_3, function_call="auto")

print(output)

{
  "symbol": "NQ",
  "timerange": "last month"
}


### Trend Visualization

In [9]:
# # in this cell we force the model to use show_trend
# messages = []

# messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
# messages.append({"role": "user", "content": "Show me the trend of E-mini Nasdaq-100(NQ) over the past two weeks"})

# chat_response = chat_completion_request(
#     messages, tools=tools, tool_choice={"type": "function", "function": {"name": "show_trend"}}, model=GPT_MODEL_3
# )

# chat_response

In [6]:
user_prompt = "Show me the trend of NQ stock which is equal to one over the last month."

output = get_response(user_prompt, functions, model=GPT_MODEL_3, function_call="auto")

print(output)

{
  "symbol": "NQ",
  "timerange": "1m",
  "trend": 1
}


### Order Entry

In [11]:
# # in this cell we force the model to use order_entry_risk_manager
# messages = []

# messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
# messages.append({"role": "user", "content": "Open a long position for me in E-mini Nasdaq-100(NQ) with order type of limit and price of 2.0 and size of 10 and stop loss of 5 and take profit of 20"})

# chat_response = chat_completion_request(
#     messages, tools=tools, tool_choice={"type": "function", "function": {"name": "order_entry_risk_manager"}}, model=GPT_MODEL_3
# )

# chat_response

In [7]:
user_prompt = "Would it be successful if I open a long position with the type of limit based on ES and price of 10.1 dollars and size of 10 and stop loss of 1.2 dollars and take profit of 5.78 dollars?"

output = get_response(user_prompt, functions, model=GPT_MODEL_3, function_call="auto")

print(output)

{
  "symbol": "ES",
  "direction": "long",
  "order_type": "limit",
  "price": "10.1 dollars",
  "size": 10,
  "sl": "1.2 dollars",
  "tp": "5.78 dollars"
}


### Calculate Support and Resistance Levels

In [13]:
# # in this cell we force the model to use calculate_sr
# messages = []

# messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
# messages.append({"role": "user", "content": "Calculate support and resistance levels of E-mini Nasdaq-100(NQ) for the past 2 days with the time frame of 2 hours"})

# chat_response = chat_completion_request(
#     messages, tools=tools, tool_choice={"type": "function", "function": {"name": "calculate_sr"}}, model=GPT_MODEL_3
# )

# chat_response

In [8]:
user_prompt = "Calculate Support and Resistance Levels based on RTY by looking back upto 15 days and timeframe of two days"

output = get_response(user_prompt, functions, model=GPT_MODEL_3, function_call="auto")

print(output)

{
  "symbol": "RTY",
  "lookback_days": "15",
  "timeframe": "2d"
}


### News Analysis 

In [15]:
# # in this cell we force the model to use news_analysis
# messages = []

# messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
# messages.append({"role": "user", "content": "What effect would the news of the past 2 weeks of federal announcement would have on E-mini Nasdaq-100(NQ)"})

# chat_response = chat_completion_request(
#     messages, tools=tools, tool_choice={"type": "function", "function": {"name": "news_analysis"}}, model=GPT_MODEL_3
# )

# chat_response

In [9]:
user_prompt = "Can you analyze the federal announcement news with time range of 2 and 4 days on CL?"

output = get_response(user_prompt, functions, model=GPT_MODEL_3, function_call="auto")

print(output)

{
  "symbol": "CL",
  "timerange": [2, 4],
  "news_type": "federal announcement"
}


## Parallel Function Calling

In [24]:
prompt_samples = [
    "",
    "",
    ""
]

In [17]:
# # in this cell we force the model to use news_analysis
# messages = []

# messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
# messages.append({"role": "user", "content": prompt_samples[0]})

# chat_response = chat_completion_request(
#     messages, tools=tools, tool_choice=tools, model=GPT_MODEL_3
# )

# chat_response